In [ ]:
# !pip install pyspark

# Índice

1. [Imports](#Import)
2. [Usando dropna](#dropna)
3. [Dropando duplicados](#dropDuplicates)
4. ...

## Import

In [1]:
# Importação das bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, concat

In [3]:
# Inicializar uma sessão do Spark e impedindo avisos desnecessários
spark = SparkSession.builder \
    .appName("NomeDaApp") \
    .config("spark.logConf", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

## dropna

In [11]:
# Exemplo de DataFrame com 10 registros e 4 campos (incluindo valores nulos/vazios)
data = [
    ("Alice", 34, "São Paulo", "Brasil"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Alice", None, "São Paulo", "Brasil"),  # Idade nula
    ("Carla", 29, "", "Portugal"),          # Cidade vazia
    ("David", 30, "Berlim", None),          # País nulo
    ("Eva", 25, "Paris", "França"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Frank", 40, "Tóquio", "Japão"),
    ("Grace", 35, None, "Reino Unido"),     # Cidade nula
    ("Hank", 50, "Sydney", "Austrália")
]

df = spark.createDataFrame(data, ["Nome", "Idade", "Cidade", "País"])

# Remover linhas com valores nulos ou vazios e salvar na mesma variável `df`
df = df.na.drop()  # Remove linhas com qualquer valor nulo ou vazio

# Mostrar o resultado
df.show()

+-----+-----+---------+---------+
| Nome|Idade|   Cidade|     País|
+-----+-----+---------+---------+
|Alice|   34|São Paulo|   Brasil|
|  Bob|   45|Nova York|      EUA|
|Carla|   29|         | Portugal|
|  Eva|   25|    Paris|   França|
|  Bob|   45|Nova York|      EUA|
|Frank|   40|   Tóquio|    Japão|
| Hank|   50|   Sydney|Austrália|
+-----+-----+---------+---------+



## dropDuplicates

In [10]:
# Exemplo de DataFrame com 10 registros e 4 campos
data = [
    ("Alice", 34, "São Paulo", "Brasil"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Alice", 34, "São Paulo", "Brasil"),
    ("Carla", 29, "Lisboa", "Portugal"),
    ("David", 30, "Berlim", "Alemanha"),
    ("Eva", 25, "Paris", "França"),
    ("Bob", 45, "Nova York", "EUA"),
    ("Frank", 40, "Tóquio", "Japão"),
    ("Grace", 35, "Londres", "Reino Unido"),
    ("Hank", 50, "Sydney", "Austrália")
]

df = spark.createDataFrame(data, ["Nome", "Idade", "Cidade", "País"])

# Remover linhas duplicadas e salvar na mesma variável `df`
df = df.dropDuplicates()

# Mostrar o resultado
df.show()

+-----+-----+---------+-----------+
| Nome|Idade|   Cidade|       País|
+-----+-----+---------+-----------+
|Alice|   34|São Paulo|     Brasil|
|  Bob|   45|Nova York|        EUA|
|David|   30|   Berlim|   Alemanha|
|Carla|   29|   Lisboa|   Portugal|
|  Eva|   25|    Paris|     França|
|Frank|   40|   Tóquio|      Japão|
|Grace|   35|  Londres|Reino Unido|
| Hank|   50|   Sydney|  Austrália|
+-----+-----+---------+-----------+

